In [ ]:
# pip install fosforml

In [1]:
# pip show fosforml

In [1]:
from fosforml.model_manager.snowflakesession import get_session
session = get_session()

In [3]:
from snowflake.snowpark.session import Session,col
import snowflake.snowpark.functions
from snowflake.ml.modeling.metrics import confusion_matrix, accuracy_score, f1_score, recall_score,precision_score
from snowflake.ml.modeling.preprocessing import LabelEncoder, StandardScaler
from snowflake.ml.modeling.impute import SimpleImputer
from snowflake.ml.modeling.ensemble import RandomForestRegressor
from snowflake.ml.modeling.pipeline import Pipeline
import snowflake

In [4]:
def apply_label_encoding(input_data_frame):
    for i in input_data_frame.dtypes:
        if i[1].find('string') >= 0:
            label_encoder = LabelEncoder(input_cols=i[0],output_cols=i[0],drop_input_cols=True)
            input_data_frame = label_encoder.fit(input_data_frame).transform(input_data_frame)
    return input_data_frame

In [5]:
import pandas as pd
df = pd.read_csv("HR_CHURN.csv")
snow_df = session.create_dataframe(df)
input_data_frame, test_df = snow_df.randomSplit([0.75, 0.25])

In [6]:
label_encoder_column = []     
input_data_frame = apply_label_encoding(input_data_frame)
feature_cols = input_data_frame.columns
target_col = "SALARY"
feature_cols.remove(target_col)
OUTPUT_COLS = [target_col + '_PREDICTION']
input_data_frame.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECTS"  |"AVERAGE_MONTHLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT_COMPANY"  |"PROMOTION_LAST_5YEARS"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1.0       |7.0           |0.38                  |0.53               |2                  |157                      |3                     |False            |True            |False                    |
|2.0       |7.0           |0.8                   |0.86               |5                  |262                      |6                     |False            |True            |False                 

In [7]:
pipeline = RandomForestRegressor(input_cols=feature_cols, label_cols=target_col)
pipeline.fit(input_data_frame)

The version of package 'snowflake-snowpark-python' in the local environment is 1.25.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.
Failed to execute query [queryID: 01b8d343-0712-581f-0072-f3031457459a] 
CREATE OR REPLACE 
TEMPORARY  PROCEDURE  SNOWPARK_TEMP_PROCEDURE_C0EEIWXOA4(arg1 ARRAY,arg2 STRING,arg3 ARRAY,arg4 ARRAY,arg5 STRING,arg6 OBJECT)

RETURNS STRING
LANGUAGE PYTHON 
VOLATILE
RUNTIME_VERSION=3.11
IMPORTS=('@"SPCS_AMIT"."SCHEMA_SPCS_AMIT".SNOWPARK_TEMP_STAGE_ZFABVOUDNO/SNOWPARK_TEMP_PROCEDURE_C0EEIWXOA4/udf_py_1804293618.zip')
PACKAGES=('snowflake-snowpark-python','numpy==1.26.4

SnowparkSQLException: (1300) (1304): 01b8d343-0712-581f-0072-f3031457459a: 391546 (XX000): SQL compilation error: Cannot create a Python function with the specified packages. Please check your packages specification and try again. 'One or more package conflicts were detected.'. Hint: These packages are available in other python runtimes:["scikit-learn==1.3.0"->[3.8, 3.9, 3.10], "snowflake-snowpark-python"->[3.8, 3.9, 3.10], "cloudpickle==2.2.1"->[3.8, 3.9, 3.10], "xgboost==1.7.6"->[3.8, 3.9, 3.10], "numpy==1.26.4"->[3.9, 3.10]].

In [9]:
pred_snow_df = pipeline.predict(apply_label_encoding(test_df))

The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.


In [10]:
from fosforml import register_model

In [ ]:
register_model(
  model_obj=pipeline,
  session=session,
  snowflake_df=pred_snow_df,
  name="HR_Churn_model",
  description="This is a test model",
  flavour="snowflake",
  dataset_source="HR_CHURN",
  dataset_name="Hr_churn",
  datasource_name="Hr_churn",
  model_type="regression",
  conda_dependencies=["scikit-learn==1.3.2"],
  source="Notebook"
)

Calculating build time metrics

Progress: ██████████████████                                                     25.0%


The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.*'. Your UDF might not work when the package version is different between the server and your local environment.


Calculating build time metrics

Progress: ███████████████████████████████████                                    50.0%
Calculating build time metrics

Progress: ████████████████████████████████████████████████████                   75.0%
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


"Failed to save build time metrics for model 'MODEL_031DBA4D_7F94_40CA_A850_EA33E610521B_FDC_HR_CHURN_MODEL'. list index out of range"